In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 43.8 MB/s eta 0:00:00


In [3]:
import os
import cv2
import mediapipe as mp
import csv
import json
import pandas as pd

In [4]:
relevant_keypoints = [
    mp.solutions.pose.PoseLandmark.NOSE,
    mp.solutions.pose.PoseLandmark.LEFT_EYE_INNER,
    mp.solutions.pose.PoseLandmark.LEFT_EYE,
    mp.solutions.pose.PoseLandmark.LEFT_EYE_OUTER,
    mp.solutions.pose.PoseLandmark.RIGHT_EYE_INNER,
    mp.solutions.pose.PoseLandmark.RIGHT_EYE,
    mp.solutions.pose.PoseLandmark.RIGHT_EYE_OUTER,
    mp.solutions.pose.PoseLandmark.LEFT_EAR,
    mp.solutions.pose.PoseLandmark.RIGHT_EAR,
    mp.solutions.pose.PoseLandmark.MOUTH_LEFT,
    mp.solutions.pose.PoseLandmark.MOUTH_RIGHT,
    mp.solutions.pose.PoseLandmark.LEFT_SHOULDER,
    mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER,
]

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [5]:
processed_frames_path = "/content/drive/MyDrive/Exercise Tracking/frames"
keypoints_output_path = "/content/drive/MyDrive/Exercise Tracking/keypoints"

os.makedirs(keypoints_output_path, exist_ok=True)

In [6]:
def extract_keypoints(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    keypoints = []
    if results.pose_landmarks:
        for landmark in relevant_keypoints:
            landmark_px = results.pose_landmarks.landmark[landmark]
            keypoints.append([landmark_px.x, landmark_px.y, landmark_px.z])
    else:
        keypoints = [[None, None, None]] * len(relevant_keypoints)

    return keypoints

In [7]:
def process_keypoints():
    for exercise_class in os.listdir(processed_frames_path):
        class_path = os.path.join(processed_frames_path, exercise_class)
        if not os.path.isdir(class_path):
            continue

        for exercise_number in os.listdir(class_path):
            exercise_path = os.path.join(class_path, exercise_number)
            if not os.path.isdir(exercise_path):
                continue

            keypoints_data = []
            for frame in os.listdir(exercise_path):
                frame_path = os.path.join(exercise_path, frame)
                keypoints = extract_keypoints(frame_path)
                flattened_keypoints = [coord for point in keypoints for coord in point]
                keypoints_data.append([frame] + flattened_keypoints)

            # Save keypoints as CSV
            keypoints_df = pd.DataFrame(
                keypoints_data,
                columns=["frame"] + [f"{lm.name}_{axis}" for lm in relevant_keypoints for axis in ["x", "y", "z"]],
            )
            output_file = os.path.join(keypoints_output_path, f"{exercise_number}_keypoints.csv")
            keypoints_df.to_csv(output_file, index=False)
            print(f"Keypoints saved for {exercise_class}/{exercise_number} at {output_file}")

In [8]:
process_keypoints()

Keypoints saved for Neck Rotation/Neck_Rotation_6 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_6_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_7 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_7_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_1 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_1_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_2 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_2_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_3 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_3_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_4 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_4_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_5 at /content/drive/MyDrive/Exercise Tracking/keypoints/Neck_Rotation_5_keypoints.csv
Keypoints saved for Neck Rotation/Neck_Rotation_8 at /content/